### Description

This kernel shows how to get to LB 0.96610 with a single QDA (version 1 has the score) in less than 1 minute.

## Building the QDA Classifier

### Loading Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler


from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

### Loading Data

In [2]:
%%time

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

CPU times: user 20.6 s, sys: 1.64 s, total: 22.2 s
Wall time: 22.2 s


### Training the Classifiers on All Data

In [3]:
%%time
preds = np.zeros(len(test))

cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]
    
# BUILD 512 SEPARATE NON-LINEAR MODELS
for i in range(512):
    print(i)
    # EXTRACT SUBSET OF DATASET WHERE WHEEZY-MAGIC EQUALS i
    train2 = train[train['wheezy-copper-turtle-magic']==i].copy()
    test2 = test[test['wheezy-copper-turtle-magic']==i].copy()
    #idx1 = train2.index
    idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    # FEATURE SELECTION (USE APPROX 40 OF 255 FEATURES)
    sel = VarianceThreshold(threshold=1.5).fit(train2[cols])
    train3 = sel.transform(train2[cols])
    test3 = sel.transform(test2[cols])

    #BUILDING THE PIPELINE FOR THE CLASSIFIER
    pipe = Pipeline([('scaler', StandardScaler()),
                    ('qda', QuadraticDiscriminantAnalysis(reg_param=0.111))
                   ])       


    pipe.fit(train3,train2['target'])
    preds[idx2] = pipe.predict_proba(test3)[:,1]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import neural_network
from sklearn.svm import NuSVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [11]:
preds2 = np.zeros(len(test))
oof = np.zeros(len(train))
for i in range(512):
    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    sel = VarianceThreshold(threshold=1.5).fit(train2[cols])
    train3 = sel.transform(train2[cols])
    test3 = sel.transform(test2[cols])

    #BUILDING THE PIPELINE FOR THE CLASSIFIER
    pipe = Pipeline([('scaler', StandardScaler()),
                    ('nusvc', NuSVC(probability=True, kernel='poly', degree=4, gamma='auto', random_state=4, nu=0.6, coef0=0.08)) 
                   ])       
    
    pipe.fit(train3,train2['target'])
    preds2[idx2] = pipe.predict_proba(test3)[:,1]
        
    if i%15==0: print(i)

0
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
480
495
510
0.5


All done! At this point we are ready to make our submission file!

In [12]:
prediction = (preds + preds2)/2

In [13]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = prediction
sub.to_csv('submission.csv',index=False)

In [14]:
sub.shape

(131073, 2)

In [15]:
sub.head()

,id,target
0,1c13f2701648e0b0d46d8a2a5a131a53,0.879656
1,ba88c155ba898fc8b5099893036ef205,0.854420
2,7cbab5cea99169139e7e6d8ff74ebb77,0.228751
3,ca820ad57809f62eb7b4d13f5d4371a0,0.263152
4,7baaf361537fbd8a1aaa2c97a6d4ccc7,0.218052
